In [1]:
import cPickle as pickle
import numpy as np
import pandas as pd
import h5py
import matplotlib.pyplot as plt
import matplotlib as mpl
import CAM_NWB as cn

%matplotlib inline

In [3]:
path = ('/Volumes/Brain2015/CAM/')
group = ('185282')
baseline_method = ('_mean_preStim_responses.npy')
traces = np.load(path + 'traces_' + group + '.npy')


In [4]:
pklfile = open(path+'CAM_Meta.pkl','r') # importing the compiled CAM meta data
CAM_Meta = pickle.load(pklfile)
pklfile.close()

In [5]:
CAM_Meta_X = CAM_Meta[(CAM_Meta.specimen==group)]
group_ids = list(CAM_Meta_X.lims_id)
len(group_ids)

7

In [6]:

tuning={}
for x, filename in enumerate(group_ids[:1]):
    print x,filename
    exp_traces = traces[x]
    baseline = np.load(path + 'baselines' + '/' + filename + baseline_method)
    baseline = baseline.T
    DFF_traces = np.nan*np.ones(exp_traces.shape)
    for counter in range(DFF_traces.shape[2]): #DF/F calculation (exp_traces/baseline)-1
        DFF_traces[:,:,counter] = (exp_traces[:,:,counter]/baseline)-1
    
    mean_response= np.mean(DFF_traces[:,:,30:90], axis=2)
    stimparams = cn.getStimulusTable(path + filename + '/' + filename + '.nwb')
    
    orientations = np.unique(stimparams['orientation'])
    orientations = orientations[~np.isnan(orientations)]

    TFs = np.unique(stimparams['temporal_frequency'])
    TFs = TFs[~np.isnan(TFs)]
    
    summary_order = ['Orientation','Temporal_Frequency','# Trials']
    stim_response = pd.DataFrame(index = range(len(orientations)*len(TFs)), 
                             columns = summary_order + range(mean_response.shape[0]))
 
    
    blank_index = stimparams[(stimparams.blank_sweep)==1].index
    blank_trials = DFF_traces[:,blank_index,:]
    blank_mean = np.mean(mean_response[:,blank_index], axis=1)
    blank_sd = np.std(mean_response[:,blank_index], axis=1)
    
    
    s=0

    for i, ori in enumerate(orientations):
        for k, tf in enumerate(TFs):
            stimpairs= stimparams[(stimparams.orientation==ori) & (stimparams.temporal_frequency==tf)]
            stimpairs_index = list(stimpairs.index)
            std_response = np.std(mean_response[:,stimpairs_index], axis=1)
            stim_response_dict = {'Orientation': ori, 'Temporal_Frequency': tf,'#Trials': len(stimpairs_index)} 
            response_dict = {counter: response for counter, response in enumerate(response)}
            stim_response_dict.update(response_dict)
            stim_response.loc[s] = pd.Series(stim_response_dict)
            s=s+1
            
    temptuning=pd.DataFrame(index=range(DFF_traces.shape[0]), columns = ['ROI'] + summary_order[:2])
    for roi in range(DFF_traces.shape[0]):
        peak = stim_response[roi].idxmax()
        ori_tune = stim_response.Orientation[peak]
        ori_orth = ori_tune-90
        tf_tune = stim_response.Temporal_Frequency[peak]
        #orth_response = stim_response
        tuning_dict = {'ROI':roi,'Orientation':ori_tune,'Temporal_Frequency':tf_tune}
        temptuning.loc[roi] = pd.Series(tuning_dict)
        
    tuning[filename] = temptuning
    
    

0 482923718
